In [1]:
import urllib.request
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [6]:
def gen_data(api_node,search_text,start_num,disp_num):
    base = 'https://openapi.naver.com/v1/search'
    node = '/' + api_node + '.json'
    param_query = '?query=' + urllib.parse.quote(search_text)
    param_start = '&start=' + str(start_num)
    param_disp = '&display=' + str(disp_num)
    
    url = base + node + param_query + param_disp + param_start
    responseDecode = get_result_onpage(url)

    if(responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [14]:
def get_result_onpage(url):
    request = urllib.request.Request(url)
    request.add_header('X-Naver-Client-Id', client_id)
    request.add_header('X-Naver-Client-Secret', client_secret)
    
    try:
        response = urllib.request.urlopen(request)
        if response.getcode() == 200:
            print("[%s]Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e :
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    return response.read().decode('utf-8')

In [12]:
def delete_tag(input_str):
    input_str = input_str.replace('<b>', '')
    input_str = input_str.replace('</b>', '')
    input_str = input_str.replace('\xa0', '')
    return input_str


def get_fields(json_data):
    title = [delete_tag(each['title']) for each in json_data['items']]
    link = [each['link'] for each in json_data['items']]
    lprice = [each['lprice'] for each in json_data['items']]
    image = [each['image'] for each in json_data['items']]
    category1 = [each['category1'] for each in json_data['items']]
    category2 = [each['category2'] for each in json_data['items']]
    category3 = [each['category3'] for each in json_data['items']]
    category4 = [each['category4'] for each in json_data['items']]
    
    result = pd.DataFrame({
        'title': title,
        'image': image,
        'link': link,
        'lprice': lprice,
        'category1' : category1,
        'category2' : category2,
        'category3' : category3,
        'category4' : category4,
        
    }, columns=['title','image','lprice', 'link','category1','category2','category3','category4'])

    return result, len(json_data)

In [15]:
client_id = 'Ks3eAGbsx4702Eicg0IO'
client_secret = 'pCPPzM3r2c'

result_datas = []
cnt = 0

json_result = gen_data('shop', '의류', 1, 100)
total = json_result['total']

while((json_result != None) and (json_result['display'] != 0)):
    result, cnt_tmp = get_fields(json_result)
    cnt += cnt_tmp
    result_datas.append(result)
    
    start = json_result['start'] + json_result['display']
    json_result = gen_data('shop', '의류', start, 100)
    
# 리스트로 저장된 데이터셋을 concat 함수로 모두 병합해주고, reset_index로 인덱스를 초기화해 0부터 정렬되도록 하자.
result_datas_concat = pd.concat(result_datas)
result_datas_concat.reset_index(drop=True, inplace=True)

# 필요하다면 csv 파일로 데이터를 저장할 수도 있다.
result_datas_concat.to_csv('./Naver_shopping.csv', sep=',', encoding="utf-8")

[2024-01-28 20:24:41.900990]Url Request Success
[2024-01-28 20:24:42.304080]Url Request Success
[2024-01-28 20:24:42.723174]Url Request Success
[2024-01-28 20:24:43.155271]Url Request Success
[2024-01-28 20:24:43.596370]Url Request Success
[2024-01-28 20:24:44.032468]Url Request Success
[2024-01-28 20:24:44.463565]Url Request Success
[2024-01-28 20:24:44.883660]Url Request Success
[2024-01-28 20:24:45.342762]Url Request Success
[2024-01-28 20:24:45.788862]Url Request Success
HTTP Error 400: Bad Request
[2024-01-28 20:24:45.834872] Error for URL : https://openapi.naver.com/v1/search/shop.json?query=%EC%9D%98%EB%A5%98&display=100&start=1001


In [21]:
df = pd.read_csv('./Naver_shopping.csv', sep=',')

TypeError: read_csv() got an unexpected keyword argument 'index'

In [17]:
df

,Unnamed: 0,title,image,lprice,link,category1,category2,category3,category4
0,0,탑텐 TOPTEN 공용 플란넬 라운지웨어 SET-UP MSC4UI3001,https://shopping-phinf.pstatic.net/main_422745...,18280,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,남성언더웨어/잠옷,잠옷/홈웨어,NaN
1,1,LE TROIS 23F W 이보영의 르투아 리얼 폭스퍼 롱구스다운,https://shopping-phinf.pstatic.net/main_448236...,28300,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,여성의류,점퍼,NaN
2,2,겨울 여자슬랙스 기모 밴딩슬랙스 일자 키작녀바지,https://shopping-phinf.pstatic.net/main_447941...,32900,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,여성의류,바지,NaN
3,3,올리브데올리브 폭스퍼 후드 다운 롱 패딩 YP3WH830,https://shopping-phinf.pstatic.net/main_431482...,108270,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,여성의류,점퍼,NaN
4,4,어니우니 20수 무지 반팔 티셔츠 기본 라운드 면티,https://shopping-phinf.pstatic.net/main_326721...,8400,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,남성의류,티셔츠,NaN
...,...,...,...,...,...,...,...,...,...
995,995,유니렉스 흰티 라운드 레이어드 무지 반팔티 흰색 면티,https://shopping-phinf.pstatic.net/main_824097...,6600,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,남성의류,티셔츠,NaN
996,996,에잇컨셉 아웃스티치 퍼피 기모맨투맨 C23K7FTS015,https://shopping-phinf.pstatic.net/main_434363...,23320,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,여성의류,티셔츠,NaN
997,997,올젠 롯데올젠 남자 겨울 면스판 치노팬츠 5종 1택 TS 672088 ZOC4PP1303,https://shopping-phinf.pstatic.net/main_428869...,26150,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,남성의류,바지,NaN
998,998,탑텐 남성 슈퍼 에어 하이넥 숏다운 점퍼 MSD4JP1902,https://shopping-phinf.pstatic.net/main_449291...,75240,https://search.shopping.naver.com/gate.nhn?id=...,패션의류,남성의류,점퍼,NaN


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   title       1000 non-null   object
 2   image       1000 non-null   object
 3   lprice      1000 non-null   int64 
 4   link        1000 non-null   object
 5   category1   1000 non-null   object
 6   category2   1000 non-null   object
 7   category3   1000 non-null   object
 8   category4   13 non-null     object
dtypes: int64(2), object(7)
memory usage: 70.4+ KB
